# Unsupervised SRBM

We can train RBM unsupervised.

Originally, we use data and model distribution to maximize the loglikelihood.

\begin{align}
{\rm Max} \left\langle \ln ( q(\phi) ) \right\rangle_{p}
\end{align}

Instead we can use generated data to estimate p.

\begin{align}
\mathcal{L} &= {\rm Max} \left\langle \ln ( p(\phi) ) \right\rangle_{q} \\
&= {\rm Max} \left\langle -S_{\phi}(\phi) - \ln Z \right\rangle
\end{align}

\begin{align}
\frac{\partial \mathcal{L}}{\partial w_{ia}} &= -\frac{\partial \langle S_{\phi}(\phi^{\rm rbm}) \rangle_{q}}{\partial w_{ia}}
 - \langle \phi_i \phi_j w_{ja} \rangle_{q} \\
 &= \left\langle S_{\phi}(\phi) \frac{\partial S_{\rm RBM}}{\partial w_{ia}} \right\rangle - \langle \phi_i \phi_j w_{ja} \rangle \\
 &= \left\langle S_{\phi} (\phi) \phi_i \phi_j w_{ja} \right\rangle - \langle \phi_i \phi_j w_{ja} \rangle
\end{align}

or

\begin{align}
\frac{\partial \mathcal{L}}{\partial w_{ia}} &= \langle \phi_i \phi_j w_{ja} \rangle_{q}
 - \langle \phi_i \phi_j w_{ja} \rangle_{q} \\
 &= K^{-1}_{\phi, ij}w_{ja} - K^{-1}_{\rm RBM, ij} w_{ja}
\end{align}

We have to know the expression for one-point and two point function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image

import random

# Change accordingly to your directory structure
sys.path.append('../')
import RBM

In [ ]:
N = 10

In [ ]:
conf_id = 'phi_1d_10N_2m'
# data_file = np.load('../datas/scalar_field/phi_1d_10N_2m_2/phi_1d_10N_2m_2.npy')
# data_file = np.load('../datas/scalar_field/'+conf_id+'/'+conf_id+'.npy')
# n_data = len(data_file)

In [ ]:
def S(field, m):
    N = len(field)
    s = 0.
    for i in range(N):
        s += -0.5*field[i]*(field[(i+1)%N]
                            + field[(i-1)%N] - (2.+ m[i]**2)*field[i])

    return s/N

def S_fast(field,m=2.):
    s = m**2 * field**2
    s += 2.*field**2
    s -= field*torch.roll(field,-1,dims=1)
    s -= field*torch.roll(field,1, dims=1)
    return torch.sum(0.5*s)/N

In [ ]:
# True matrix
m = 2
W = np.zeros((N,N))

for i in range(N):
    for j in range(N):
        if i==j:
            W[i][j] = 2 + m**2
#             W[i][j] = 2.
        elif (i % N == (j+1) %N) or (i % N == (j-1) %N):
            W[i][j] = -1
print(W)
K_phi = W

$w w^T = -K_{\phi} + m^2$

In [ ]:
# Cholesky solution
W_phi = np.linalg.cholesky(-K_phi + np.diag([5**2]*N))

In [ ]:
-K_phi + np.diag([5**2]*N)

In [ ]:
W_phi @ W_phi.T

In [ ]:
plt.imshow(W_phi, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
W_phi

In [ ]:
N = 10
# Initialization scheme.
# Initialize w with Cholesky solution W_phi and mass=5 sigma=1
# init_cond = {'w':torch.DoubleTensor(W_phi.copy()),'m':5., 'sig':1.}
# init_cond = {'m':3., 'sig':1., 'm_scheme':'local'}
init_cond = {'m':3., 'sig':1., 'm_scheme':'global'}

# Reproducibility
torch.manual_seed(1234)
random.seed(1234)
np.random.seed(1234)

# Initialize SRBM
rbm = RBM.SRBM(n_v=N,n_h=N,k=10,init_cond=init_cond)

# For autograd if implimented
# train_op = optim.SGD(rbm.parameters(),1e-7)
# train_op = optim.Adam(rbm.parameters(),1e-5)

# Check initial coupling matrix
print(rbm.w)

# Training parameters
lr = 1e-1
epochs = 3000
batch_size = 64

# Initial field.
# Cold start
init_field = torch.ones((batch_size,N))

# wrap K_phi to torch
K_phi_tc = torch.DoubleTensor(K_phi)

In [ ]:
# Save and load model
rbm.save('./')
saved_model = rbm.name
print(saved_model)
rbm = RBM.SRBM(load='./'+saved_model+'.npz')

In [ ]:
!rm *.npz

In [ ]:
# Train the model
history = rbm.unsup_fit(K_phi_tc, S_fast, epochs, lr, batch_size, verbose=True, lr_decay=0.99)

# Training result

In [ ]:
plot_dir = '../plots/'
model_name ='normal'

In [ ]:
# Loss function
plt.plot(np.arange(len(history['loss'])),history['loss'])
plt.title('KL')
plt.xlabel('epoch')
plt.ylabel(r'$\mathcal{L}$')
# plt.savefig(plot_dir+model_name+'_lc.jpg')
plt.show()

In [ ]:
# Gradient
plt.plot(np.arange(len(history['dw'])),np.mean(np.mean(history['dw'], axis=1),axis=1), label='mean')
plt.plot(np.arange(len(history['dw'])),np.min(np.min(history['dw'], axis=1),axis=1), label='min')
plt.plot(np.arange(len(history['dw'])),np.max(np.max(history['dw'], axis=1),axis=1), label='max')
plt.title('dW')
plt.xlabel('epoch')
plt.ylabel(r'$\frac{d L}{dw}$')
plt.legend()
# plt.savefig(plot_dir+model_name+'_lc.jpg')
plt.show()

In [ ]:
n_samples = 1000
recon_field = torch.ones((n_samples, N))
v_pred, v_, h_pred, _, v = rbm.forward(recon_field,n_samples)
n_samples = len(v)
# S_true = np.zeros(n_data)
S_pred = np.zeros(n_samples)
S_K = np.zeros(n_samples)
M = m*np.ones(N)

# for i in range(n_data):
#     S_true[i] = S(data_file[i],M)

for i in range(n_samples):
    S_pred[i] = S(v_.data.numpy()[i],M)
    S_K[i] = -rbm.free_energy(v_[i:i+1]).data.numpy()
#     w_ = rbm.W.data.numpy().copy()
#     m_ = rbm.mass.data.numpy().copy()
#     sig_ = rbm.h_std
#     Kin = -sig_**2 *(w_.T @ w_)
#     Mss = np.diag(m_**2)m
#     s_ = 0.5 * v_pred.data.numpy()[i] @ (Kin + Mss) @ v_pred.data.numpy()[i].T/N
#     S_K[i] = s_

plt.hist(S_K, bins=50, density=True, color='C2', label='Model', alpha=0.8)
plt.hist(S_pred, bins=50, density=True, color='C1', label='Reconstructed', alpha=0.8)    
# plt.hist(S_true, bins=50, density=True, color='C0', label='MCMC data', alpha=0.8)
plt.legend()
plt.title('Action histogram')
# plt.savefig(plot_dir+model_name+'_S.jpg')
plt.show()

In [ ]:
np.mean(S_pred)

In [ ]:
np.std(S_pred)

In [ ]:
# SVD of coupling matrix squared
s_hist = np.zeros((epochs,N))
for i in range(epochs):
    _, s_, _ = np.linalg.svd(history['w'][i])
    s_hist[i] = s_

plt.plot(np.arange(epochs),s_hist**2)
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$w_{\alpha}^2$')
plt.title(r'$w^2$ evolution')
# plt.savefig(plot_dir+model_name+'_w.jpg')
plt.show()

In [ ]:
# Last few steps
plt.plot(np.arange(epochs)[-10:],s_hist[-10:]**2, '.-')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$w_{\alpha}^2$')
plt.title(r'$w^2$ evolution')
# plt.savefig(plot_dir+model_name+'_w.jpg')
plt.show()

In [ ]:
# SVD of coupling matrix not squared
plt.plot(np.arange(epochs),s_hist, '-')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$w_{\alpha}$')
plt.title(r'$w_{\alpha}$ evolution')
# plt.savefig(plot_dir+model_name+'_w.jpg')
plt.show()

In [ ]:
# Last few steps
plt.plot(np.arange(epochs)[-10:],s_hist[-10:], '.-')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$w_{\alpha}$')
plt.title(r'$w_{\alpha}$ evolution')
# plt.savefig(plot_dir+model_name+'_w.jpg')
plt.show()

In [ ]:
# Kernel SVD values
s_hist = np.zeros((epochs,N))
mu2 = np.diag(np.ones(N))

for i in range(epochs):
    WW_ = history['w'][i].T@history['w'][i]
    K_ = -rbm.sig**2 * WW_ + np.diag(history['m'][i]**2)
    if i ==0:
        K_i = K_.copy()
    s_ = np.sort(np.linalg.eigvals(K_))
    s_hist[i] = s_

plt.plot(np.arange(epochs),s_hist)
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$K_{\alpha}$')
plt.title('K eigenvalue')
# plt.savefig(plot_dir+model_name+'_K.jpg')
plt.show()

In [ ]:
# Last few steps
plt.plot(np.arange(epochs)[-10:],s_hist[-10:], '.-')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'$K_{\alpha}$')
plt.title('K eigenvalue')
# plt.savefig(plot_dir+model_name+'_K.jpg')
plt.show()

In [ ]:
# Mass parameter of the model
plt.plot(np.arange(len(history['m'])),history['m'])
plt.axhline(np.sqrt(m**2 + 2. + 2.), ls='--', color='C3', label='Minimum Cholesky mass limit')
plt.title('Mass evolution')
plt.xlabel('epoch')
plt.ylabel('mass')
plt.legend()
plt.grid(True)
# plt.savefig(plot_dir+model_name+'_mass.jpg')
plt.show()

In [ ]:
# Last few steps
plt.plot(np.arange(len(history['m']))[-10:],history['m'][-10:], '.-')
# plt.axhline(np.sqrt(m**2 + 2. + 2.), ls='--', color='C3', label='Minimum Cholesky mass limit')
plt.title('Mass evolution')
plt.xlabel('epoch')
plt.ylabel('mass')
# plt.legend()
plt.grid(True)
# plt.savefig(plot_dir+model_name+'_mass.jpg')
plt.show()

In [ ]:
# K_rbm off diagonal part
Kin = (-rbm.sig**2 * (rbm.w.t() @ rbm.w)).data.numpy()
Mss = np.diag((rbm.m**2).data.numpy())
K = Kin + Mss

K_off = K - np.diag(np.diag(K))
plt.imshow(K_off, cmap='gray', vmax=K_off.max(), vmin=K_off.min())
plt.colorbar()
plt.title('K off diagonal')
# plt.savefig(plot_dir+model_name+'_K_img.jpg')
plt.show()

In [ ]:
# Coupling matrix as image
w_rbm = rbm.w.data.numpy()
plt.imshow(w_rbm, cmap='gray')
plt.colorbar()
plt.title('W')
# plt.savefig(plot_dir+model_name+'_K_img.jpg')
plt.show()

In [ ]:
# Coupling matrix off-diagonal
w_rbm = rbm.w.data.numpy()
plt.imshow(w_rbm - np.diag(np.diag(w_rbm)), cmap='gray')
plt.colorbar()
plt.title('W off-diagonal part')
# plt.savefig(plot_dir+model_name+'_K_img.jpg')
plt.show()

In [ ]:
# Difference between Choleski(initial) solution to the trained solution
diff = W_phi - w_rbm
plt.imshow(diff, cmap='gray')
plt.colorbar()
plt.show()

print('mean: ',np.mean(diff))
print('std: ',np.std(diff))

In [ ]:
# Evolution of det(ww^T)
det_hist = np.zeros(epochs)
for i in range(epochs):
    det = np.linalg.det(history['w'][i].T @ history['w'][i])
    det_hist[i] = det

plt.plot(np.arange(epochs),det_hist,'C9')
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel(r'det($ww$)')
plt.title(r'det(ww) evolution')
# plt.savefig(plot_dir+model_name+'_w.jpg')
plt.show()

In [ ]:
plt.plot(np.arange(len(history['S'])),history['S'])
plt.title('Action density')
plt.xlabel('epoch')
plt.ylabel(r'$\mathcal{s}$')
plt.grid(True)
plt.show()

# Scaling of K_RBM

In [ ]:
# Least square fit
def loss(K, K_phi, Zk, b, r):
    return np.sum((K - Zk*K_phi - b - r)**2)

def dL_dZ(K, K_phi, Zk, b, r):
    return -np.sum(2.*(K - Zk*K_phi - b - r)*K_phi)

def dL_db(K, K_phi, Zk, b, r):
    return -np.sum(2.*(K - Zk*K_phi - b - r))

def dL_dr(K, K_phi, Zk, b, r):
    return -2.*(K - Zk*K_phi - b - r)

def get_scale(model, verbose=True):
    Kin = (-model.sig**2 * (model.w.t() @ model.w)).data.numpy()
    Mss = np.diag((model.m**2).data.numpy())
    K = Kin + Mss
    mu_ = model.m.data.numpy()
    Zk_gd = mu_.min()**2/(2.**2 + 4)

    np.random.seed(1234)
    r_seed = np.random.normal(0.,.01,size=K.shape) # ~ sigma^2
    r_gd = r_seed.T @ r_seed
    b_gd = 0.
    lr = 1e-5

    for i in range(10000):
        l = loss(K, K_phi, Zk_gd, b_gd, r_gd)
    #     print(l)
        dZ = lr * dL_dZ(K, K_phi, Zk_gd, b_gd, r_gd)
        db = lr * dL_db(K, K_phi, Zk_gd, b_gd, r_gd)
        dr = lr * dL_dr(K, K_phi, Zk_gd, b_gd, r_gd)

        Zk_gd -= dZ
        b_gd -= db
        r_gd -= dr
        if i%1000 == 0 and verbose:
            print(i, l, Zk_gd, b_gd)
            
    return Zk_gd, b_gd, r_gd, K

In [ ]:
Zk, b, r, K = get_scale(rbm)

In [ ]:
Zk

# Compare Kernel

In [ ]:
# Trained Kernel
plt.title(r"$K_{\rm RBM}$")
plt.imshow(K, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
# Trained Kernel
plt.title(r"$K_{\phi}$")
plt.imshow(K_phi, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
# Difference between Choleski(initial) solution to the trained solution
diff = K_phi - K
plt.imshow(diff, cmap='gray')
plt.colorbar()
plt.show()

print('mean: ',np.mean(diff))
print('std: ',np.std(diff))

# Compare two-point correlator

In [ ]:
n_samples = 1000
init_field = torch.ones((n_samples,N))
regen, regen_, _, _, in_data = rbm.forward(init_field,10)

In [ ]:
C_phi = np.linalg.inv(K_phi)
C_rbm_kernel = np.linalg.inv(K)
C_rbm_sample = np.cov(regen_.data.numpy().copy().T)

In [ ]:
plt.title('True correlation matrix')
plt.imshow(C_phi, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
plt.title('Trained correlation matrix (Kernel)')
plt.imshow(C_rbm_kernel, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
plt.title('Trained correlation matrix (samples)')
plt.imshow(C_rbm_sample, cmap='gray')
plt.colorbar()
plt.show()